In [1]:
import os
import sys
sys.path.append('/kaggle/input/timm-0-6-9/pytorch-image-models-master')
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
from helper import *
tqdm.pandas()
from albumentations.pytorch import ToTensorV2
import albumentations as A
import timm
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from torch import nn
import torch
from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool
import random
import cv2
from math import ceil
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
tqdm.pandas()
from glob import glob
print('Done 1')

Done 1


In [2]:
%%time
helmet_data= pd.read_csv('./test_baseline_helmets.csv')
tracking_data = pd.read_csv('./test_player_tracking.csv')
labels = pd.read_csv('./sample_submission.csv')

CPU times: user 36.4 ms, sys: 12.1 ms, total: 48.5 ms
Wall time: 52.8 ms


In [3]:
%%time
label_exp = expand_contact_id(labels)
use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa',
    'team', 'position'
]
combined_df = create_features(label_exp, tracking_data, use_cols=use_cols)

CPU times: user 204 ms, sys: 36.1 ms, total: 240 ms
Wall time: 241 ms


In [4]:
def add_bboxes_details(frame, player1, player2, game_play, view):
    '''
    takes each row in comined_df and adds the required bbox information
    (may need more processing to convert to correct datatype)
    '''
    bboxes = []
    window = 24
    query = f"frame >= {frame-window}  and frame <= {frame+window} and game_play == '{game_play}'"
    if player2 == 'G':
        query += f"  and nfl_player_id in ({player1}, 'G')"
    else:
        query += f"  and nfl_player_id in ({player1}, {player2})"
    filt = helmet_data.query(query)

    tmp = filt[filt["view"] == view]
    tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
    tmp['centre_x'] = tmp['left'] + tmp['width']/2
    tmp['centre_y'] = tmp['top'] + tmp['height']/2
    frame_list = tmp.index # these are the available frames
    for fr in range(frame - window, frame + window + 1):
        if fr in frame_list:
            x, y = tmp.loc[fr][['centre_x','centre_y']]
            bboxes.append([x, y])
        else:
            bboxes.append([np.nan, np.nan])
    
    bboxes = pd.DataFrame(np.array(bboxes)).interpolate(limit_direction='both').values
    return np.array(bboxes)
   

In [5]:
combined_df['bbox_endzone'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Endzone'),
axis=1
    )
combined_df['bbox_sideline'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Sideline'),
axis=1
    )

100%|██████████| 6631/6631 [01:13<00:00, 89.72it/s] 


In [44]:
def read_image(path, cx, cy, view, aug):
    img_new = np.zeros((1, 256, 256), dtype=np.float32)
    # print(path)
    if os.path.isfile(path):
        if view == 'Endzone':
            img = cv2.imread(path, 0)[
                cy-76:cy+180, cx-128:cx+128].copy()
            img_new[0, :img.shape[0], :img.shape[1]] = img
        else:
            img = cv2.imread(path, 0)[
                cy-128:cy+128, cx-128:cx+128].copy()
            img_new[0, :img.shape[0], :img.shape[1]] = img

    return aug(image=img_new.transpose(1, 2, 0))['image'][0]

In [153]:
CFG = {
    'seed': 42,
    'num_workers':0,
    'batch_size': 1,
}

In [114]:
valid_aug = A.Compose([
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])
categorical_data_for_fitting = [
    ['home', 'CB', 'home', 'CB', ] ,
    ['home', 'DE', 'home', 'DE', ],
    ['home', 'FS', 'home', 'FS', ],
    ['home', 'TE', 'home', 'TE', ] ,
    ['home', 'ILB', 'home', 'ILB', ],
    ['home', 'OLB', 'home', 'OLB', ],
    ['home', 'T', 'home', 'T', ],
    ['home', 'G', 'home', 'G', ] ,
    ['home', 'C', 'home', 'C', ] ,
    ['home', 'QB', 'home', 'QB', ],
    ['home', 'WR', 'home', 'WR', ],
    ['home', 'RB', 'home', 'RB', ],
    ['home', 'NT', 'home', 'NT', ],
    ['home', 'DT', 'home', 'DT', ],
    ['home', 'MLB', 'home', 'MLB', ],
    ['home', 'SS', 'home', 'SS', ] ,
    ['home', 'OT', 'home', 'OT', ],
    ['home', 'LB', 'home', 'LB', ],
    ['home', 'OG', 'home', 'OG', ] ,
    ['home', 'SAF', 'home', 'SAF', ],
    ['home', 'DB', 'home', 'DB', ] ,
    ['home', 'LS', 'home', 'LS', ] ,
    ['home', 'K', 'home', 'K', ],
    ['home', 'P', 'home', 'P', ],
    ['home', 'FB', 'home', 'FB', ] ,
    ['home', 'S', 'home', 'S', ],
    ['home', 'DL', 'Ground', 'DL', ],
    ['away', 'HB', 'away', 'HB', ],
    ['away', 'HB', 'away', 'Ground', ],
]
    
one_hot = OneHotEncoder()
one_hot.fit(categorical_data_for_fitting)

OneHotEncoder()

In [194]:
class MyDataset1(Dataset):
    def __init__(self, df, aug, one_hot_transform, train=False, feature_cols=['rel_pos_x',
     'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
     'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
     'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
     'G_flug', 'orientation_1', 'orientation_2']):

        self.df = df
        self.features = feature_cols
        self.aug = aug
        self.train = train
        self.one_hot_transform = one_hot_transform

    def __len__(self):
        return ceil(len(self.df)/CFG['batch_size'])

    def get_rows(self, lnum, unum):
        lnum = lnum % len(self.df)
        unum = unum % len(self.df)

        if lnum < unum:
            return self.df[lnum:unum]
        else:
            if self.train:
                return pd.concat([self.df[lnum:], self.df[:unum]])
            else:
                return self.df[lnum:]

    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features

    def __getitem__(self, idx):
        window = 24
        frames_to_skip = 4

        row = self.get_rows(
            idx*(CFG['batch_size']), (idx+1)*(CFG['batch_size'])).reset_index(drop=True)
        
        mid_frame = row['frame']
        label = np.array(row['contact']).astype(np.float32)
        args = []
        for i in range(len(row)):
            for view in ['Endzone', 'Sideline']:
                video = row.iloc[i]['game_play'] + f'_{view}.mp4'
                cur_mid_frame = mid_frame.iloc[i]
                frames = [cur_mid_frame - window +
                          i for i in range(0, 2*window+1, frames_to_skip)]
                bbox_col = 'bbox_endzone' if view == 'Endzone' else 'bbox_sideline'
                bboxes = row.iloc[i][bbox_col][::frames_to_skip].astype(
                    np.int32)
#                 print(bboxes)

                if bboxes.sum() <= 0:
                    args += [('dummy', 0, 0, view, self.aug)]*len(frames)
                    continue

                for j, frame in enumerate(frames):
                    cx, cy = bboxes[j]
                    path = f'./work/test_frames/{video}_{frame:04d}.jpg'
                    args.append((path, cx, cy, view, self.aug))
        imgs = []
        for arg in args:
            imgs.append(read_image(*arg))

        # if __name__ == '__main__':
        #     with Pool(CFG['num_workers']) as pool:
        #         imgs = list(pool.starmap(read_image, args))
        #         pool.close()
        
        img = torch.stack(imgs).reshape(len(row), 26, 256, 256)
        features = np.array(row[self.features], dtype=np.float32)
        # print(features) same till here
        # print(row)
        features[np.isnan(features)] = 0

        """
        rel_pos_x                0
        rel_pos_y                1
        rel_pos_mag              2
        rel_pos_ori              3
        rel_speed_x              4
        rel_speed_y              5
        rel_speed_mag            6
        rel_speed_ori            7
        rel_acceleration_x       8
        rel_acceleration_y       9
        rel_acceleration_mag     10
        rel_acceleration_ori     11 
        """
        for i in range(len(row)):
            if row.iloc[i]['G_flug']:
                features[i, 6] = row.iloc[i]['speed_1']
                features[i, 7] = row.iloc[i]['direction_1']
                features[i, 10] = row.iloc[i]['acceleration_1']
                features[i, 11] = row.iloc[i]['direction_1']

                features[i, 4] = row.iloc[i]['speed_1'] * \
                    np.sin(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 5] = row.iloc[i]['speed_1'] * \
                    np.cos(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 8] = row.iloc[i]['acceleration_1'] * \
                    np.sin(row.iloc[i]['direction_1']*np.pi/180)
                features[i, 9] = row.iloc[i]['acceleration_1'] * \
                    np.cos(row.iloc[i]['direction_1']*np.pi/180)
            # print("unnormalized",features)
            features[i, :] = self.normalize_features(features[i])
        # print(features)

        team_pos = np.array(
            row[['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform(team_pos).toarray()
        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), torch.as_tensor(label)


In [175]:
video2helmets = {}
test_helmets_new = helmet_data.set_index('video')
for video in tqdm(helmet_data.video.unique()):
    video2helmets[video] = test_helmets_new.loc[video].reset_index(drop=True)

100%|██████████| 4/4 [00:00<00:00, 796.00it/s]


In [176]:
video2frames = {}
test_video_metadata = pd.read_csv("./test_video_metadata.csv")
for game_play in tqdm(test_video_metadata.game_play.unique()):
    for view in ['Endzone', 'Sideline']:
        video = game_play + f'_{view}.mp4'
        video2frames[video] = max(list(map(lambda x:int(x.split('_')[-1].split('.')[0]), \
                                           glob(f'./work/test_frames/{video}*'))))

100%|██████████| 2/2 [00:00<00:00, 239.05it/s]


In [201]:
class MyDataset2(Dataset):
    def __init__(self, df, aug=valid_aug, mode='test', transform=one_hot):
        self.df = df
        self.frame = df.frame.values
        feature_cols = ['rel_pos_x','rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
            'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x', 'rel_acceleration_y', 'rel_acceleration_mag',
            'rel_acceleration_ori', 'G_flug', 'orientation_1', 'orientation_2']
        self.feature = df[feature_cols].fillna(0).values
        self.players = df[['nfl_player_id_1','nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode
        self.one_hot_transform = transform
        
    def __len__(self):
        return len(self.df)
    
    # @lru_cache(1024)
    # def read_img(self, path):
    #     return cv2.imread(path, 0)
    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features
   
    def __getitem__(self, idx):   
        window = 24
        frame = self.frame[idx]
        
#         if self.mode == 'train':
#             frame = frame + random.randint(-6, 6)

        players = []
        for p in self.players[idx]:
            if p == 'G':
                players.append(p)
            else:
                players.append(int(p))
        
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = self.game_play[idx] + f'_{view}.mp4'

            tmp = video2helmets[video]
#             tmp = tmp.query('@frame-@window<=frame<=@frame+@window')
            tmp[tmp['frame'].between(frame-window, frame+window)]
            tmp = tmp[tmp.nfl_player_id.isin(players)]#.sort_values(['nfl_player_id', 'frame'])
            tmp_frames = tmp.frame.values
            tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
#0.002s

            bboxes = []
            for f in range(frame-window, frame+window+1, 1):
                if f in tmp_frames:
                    x, w, y, h = tmp.loc[f][['left','width','top','height']]
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([np.nan, np.nan, np.nan, np.nan])
            bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
            bboxes = bboxes[::4]

            if bboxes.sum() > 0:
                flag = 1
            else:
                flag = 0
#0.03s
                    
            for i, f in enumerate(range(frame-window, frame+window+1, 4)):
                img_new = np.zeros((256, 256), dtype=np.float32)

                if flag == 1 and f <= video2frames[video]:
                    # print(f'/kaggle/work/frames/{video}_{f:04d}.jpg')
                    img = cv2.imread(f'./work/test_frames/{video}_{f:04d}.jpg', 0)

                    x, w, y, h = bboxes[i]
                    if view == 'Endzone':
                        img = img[int(y+h/2)-76:int(y+h/2)+180,int(x+w/2)-128:int(x+w/2)+128].copy()
                    else:
                        img = img[int(y+h/2)-128:int(y+h/2)+128,int(x+w/2)-128:int(x+w/2)+128].copy()
                    img_new[:img.shape[0], :img.shape[1]] = img
                    
                imgs.append(img_new)

        
        features = np.float32(self.feature[idx])
        print(features.shape)
        # print("this is i: ",i)
        if self.df.iloc[idx]['G_flug']:
                features[ 6] = self.df.iloc[idx]['speed_1']
                features[7] = self.df.iloc[idx]['direction_1']
                features[10] = self.df.iloc[idx]['acceleration_1']
                features[11] = self.df.iloc[idx]['direction_1']

                features[4] = self.df.iloc[idx]['speed_1'] * \
                    np.sin(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[5] = self.df.iloc[idx]['speed_1'] * \
                    np.cos(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[8] = self.df.iloc[idx]['acceleration_1'] * \
                    np.sin(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[9] = self.df.iloc[idx]['acceleration_1'] * \
                    np.cos(self.df.iloc[idx]['direction_1']*np.pi/180)
        # print("unnormalized",features)
        features = self.normalize_features(features)
        # print(features)

        team_pos = np.array(
            self.df.iloc[idx][['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform([team_pos]).toarray()[0]

        img = np.array(imgs).transpose(1, 2, 0)    
        img = self.aug(image=img)["image"]
        label = np.float32(self.df.contact.values[idx])

        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), label

In [202]:
test1 = MyDataset1(combined_df, aug=valid_aug, train=False, one_hot_transform=one_hot)
test2 = MyDataset2(combined_df, aug=valid_aug, transform=one_hot)

i = 13
img1, feat1, label1 = test1[i]
img2, feat2, label2 = test2[i]

print(feat1)
print(feat2)


(15,)
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0028, -0.0095,  0.0099,  0.4538,
          0.0048, -0.0161,  0.0168,  0.4538,  0.0100,  0.2519,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0028, -0.0095,  0.0099,  0.4538,
         0.0048, -0.0161,  0.0168,  0.4538,  0.0100,  0.2519,  0.0000,  0.0000,
         1.0000,  0.0000,  0.0000,  0.0000,  0.00

In [160]:
class Validator1():
    def __init__(self, df, aug, transform, verbose=True):
        self.test_set = MyDataset1(
            df, aug=aug, train=False, one_hot_transform=transform)
        self.verbose = verbose
        print(df.iloc[-1])
        # self.criterion = criterion

    def validate(self, model):
        y_hat = []
        # y = []
        # loss = 0
        # logger.debug("converting testloader to iter")
        # logger.debug("done converting testloader to iter")
        model.eval()

        print(f"ceil: {ceil(len(self.test_set.df)/CFG['batch_size'])}")
        with torch.no_grad():
            for iteration in tqdm(range(ceil(len(self.test_set.df)/CFG['batch_size']))):

                imgs, features, labels = self.test_set[iteration]
                imgs = imgs.to(0, non_blocking=True)
                features = features.to(0, non_blocking=True)
                print(features[-1])
                # labels = labels.to(0, non_blocking=True)

                preds = model(imgs, features)
                print(preds)

                # loss += self.criterion(preds,
                #                        labels).cpu().detach().numpy().ravel()[0]
                # y.append(labels.cpu().detach().numpy())
                y_hat.append(preds.cpu().detach().numpy())

            # logger.debug(f"Combined val labels:\n{y}")
            # logger.debug(f"Combined val preds:\n{y_hat}")
            # y = np.hstack(y)
            y_hat = np.hstack(y_hat)
            # logger.debug(f"Combined val labels:\n{y}")
            # logger.debug(f"Combined val preds:\n{y_hat}")

            # loss = loss/CFG['batch_size']

            # threshs = np.linspace(0.001, 0.999, 10000)

            # mat_cors = []
            # best = 0
            # best_thresh = -1
            # for thresh in threshs:
            #     _, val_mathew_corr,  _, _, _ = get_stats(
            #         loss, y, y_hat, cur_iter=f"Val", thresh=thresh)
            #     mat_cors.append(val_mathew_corr)
            #     if val_mathew_corr > best:
            #         best = val_mathew_corr
            #         best_thresh = thresh
            # tb.add_scalar("Val Loss", loss, iteration)
            # tb.add_scalar("Val  Accuracy", val_acc,
            #               iteration)
            # tb.add_scalar("Val Mathew Correlation",
            #               val_mathew_corr, iteration)
            # if self.verbose:
            #     logger.info(f"{stats}")

        return y_hat #mat_cors, threshs, best, best_thresh

In [161]:
class Validator2():
    def __init__(self, df, aug, transform, verbose=True):
        self.test_set = MyDataset2(
            df, aug=aug, transform=transform)
        self.test_loader = DataLoader(self.test_set, batch_size=CFG['batch_size'], shuffle=False,\
                                      num_workers=CFG['num_workers'], pin_memory=True)
        self.verbose = verbose
        self.counter = 0
        print(df.iloc[-1])

    def validate(self, model):
        # logger.debug("converting testloader to iter")
        # logger.debug("done converting testloader to iter")
        model.eval()
        y_pred = []

        print(len(self.test_loader))
        with torch.no_grad():
            tk = tqdm(self.test_loader, total=len(self.test_loader))
            for step, batch in enumerate(tk):
                img, feature, label = [x.to(0) for x in batch]
                print(feature[-1])
                output = model(img, feature).squeeze(-1)
                print(output)
                y_pred.extend(output.cpu().numpy())

        y_pred = np.array(y_pred)
        return y_pred

In [132]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = timm.create_model(
            'resnet18', pretrained=False, num_classes=250, in_chans=26)
        self.mlp = nn.Sequential(
            nn.Linear(77, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.fc = nn.Linear(64+250, 1)

    def forward(self, img, feature):
        img = self.backbone(img)
        feature = self.mlp(feature)
        y = torch.sigmoid(self.fc(torch.cat([img, feature], dim=1)))
        return y.flatten()

In [133]:
model = torch.load('./model_checkpoints/baseline_3_2023-02-06-08-19-29/best_model.pth')
model.eval()

Model(
  (backbone): ResNet(
    (conv1): Conv2d(26, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [156]:
valer1 = Validator1(combined_df[1:2], aug=valid_aug, transform=one_hot)

contact_id                                         58168_003392_0_37084_G
contact                                                                 0
game_play                                                    58168_003392
step                                                                    0
nfl_player_id_1                                                     37084
nfl_player_id_2                                                         G
x_position_1                                                         41.9
y_position_1                                                        20.08
speed_1                                                              0.54
distance_1                                                           0.06
direction_1                                                        252.69
orientation_1                                                      262.31
acceleration_1                                                       0.92
sa_1                                  

In [157]:
valer2 =  Validator2(combined_df[1:2], aug=valid_aug, transform=one_hot)

contact_id                                         58168_003392_0_37084_G
contact                                                                 0
game_play                                                    58168_003392
step                                                                    0
nfl_player_id_1                                                     37084
nfl_player_id_2                                                         G
x_position_1                                                         41.9
y_position_1                                                        20.08
speed_1                                                              0.54
distance_1                                                           0.06
direction_1                                                        252.69
orientation_1                                                      262.31
acceleration_1                                                       0.92
sa_1                                  

In [158]:
y_pred1 = valer1.validate(model)

ceil: 1


100%|██████████| 1/1 [00:00<00:00, 12.18it/s]

[[   nan    nan    nan    nan    nan    nan    nan    nan    nan    nan
     nan    nan   1.   262.31    nan]]
tensor([ 0.0000,  0.0000,  0.0000,  0.0000, -0.0052, -0.0016,  0.0054,  0.7019,
        -0.0088, -0.0027,  0.0092,  0.7019,  0.0100,  0.7286,  0.0000,  1.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000], device='cuda:0')
tensor([0.8157], device='cuda:0')


In [159]:
y_pred2 = valer2.validate(model)

1


  0%|          | 0/1 [00:00<?, ?it/s]

[  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     1.   262.31   0.  ]


IndexError: single positional indexer is out-of-bounds

In [92]:
y_pred1[:3], y_pred2[:3],

(array([0.67256945, 0.81572026, 0.12307185], dtype=float32),
 array([0.67256945, 0.8845575 , 0.12307185], dtype=float32))

In [76]:
y_fin1 = y_pred1 > 0.5
y_fin2 = y_pred2 > 0.5


In [77]:
np.sum(y_fin1 != y_fin2)

2597

In [78]:
y_fin2.shape

(6631,)